In [1]:
import pyreadr, pickle, csv
import numpy as np
import pandas as pd


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

In [2]:
# This cell is an experiment in training currciulumn
# The idea is to specifically train networks to learn one class only
# To do this we will train it head on against one other class ata a time
# When trainign is complete the network will have been exposed to the desired class at least 44 times
# It will see all non-desired classes at least once
# In this instantiation the network will not be weighted ahead of time to know what class is the desired class

class Net(nn.Module):
    def __init__(self, input_size, num_classes,
                 h1_size, h2_size, h3_size, dropout_prob):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, h1_size)
        self.fc4 = nn.Linear(h1_size, num_classes)
        self.dropout = nn.Dropout(p=dropout_prob)

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = self.fc4(x)
        return nn.functional.log_softmax(x, dim=1)


# Step 1: Prepare training data
def head2head(train_y, train_X, desired, noise):
    '''This function will take a desired category and a noise category and create a paired down dataset
    The paired down data set will be an equal mix of desired and noise
    If there are not enough noise examples, a random set will be duplicated to make the lengths match
    If there are too many noise samples, a random set will be excluded'''

    # Isolate the desired instances
    TS_desired_y = train_y[train_y == desired]
    TS_desired_X = train_X[train_y == desired]
    TS_noise_y = train_y[train_y == noise]
    TS_noise_X = train_X[train_y == noise]
    # Check if the desired class is longer, if so balance it with replicates from te noise
    if len(TS_desired_y) > len(TS_noise_y):
        delta = len(TS_desired_y) - len(TS_noise_y)
        indice = np.random.choice( range(len(TS_noise_y)), delta)
        indice = torch.tensor(indice)
        extra_y = TS_noise_y[indice]
        extra_X = TS_noise_X[indice]
        TS_noise_y = torch.cat((TS_noise_y,extra_y))
        TS_noise_X = torch.cat((TS_noise_X,extra_X))
    # Check if the desired class is shorter, if so randomely discard examples from the other
    elif len(TS_desired_y) < len(TS_noise_y):
        indice = np.random.choice( range(len(TS_noise_y)), len(TS_desired_y))
        indice = torch.tensor(indice)
        TS_noise_y = TS_noise_y[indice]
        TS_noise_X = TS_noise_X[indice]
    # Combine the two balanced data sets
    TS_y = torch.cat((TS_noise_y,TS_desired_y))
    TS_X = torch.cat((TS_noise_X,TS_desired_X))
    # Shuffle the data
    indice = np.random.choice( range(len(TS_y)), len(TS_y))
    indice = torch.tensor(indice)
    TS_y = TS_y[indice]
    TS_X = TS_X[indice]
    return TS_y, TS_X


def DesiredGeneCurriculum(train_y, train_X, desired=1):
    if desired!=None:
        # Create a training set using this function to create my desired class biased training set
        noise_inds = [i for i in np.unique(train_y) if i != desired]

    # Move the data to the GPU
    # Set the device to use (GPU if available, otherwise CPU)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Convert the input data to the appropriate data type for the GPU
    train_X = train_X.to(device)
    train_y = train_y.to(device)

    ts_datasets = []
    if desired!=None:
        for noise_ind in noise_inds:
            TS_y, TS_X = head2head(train_y, train_X, desired=desired, noise=noise_ind)
            ts_datasets.append(torch.utils.data.TensorDataset(TS_X, TS_y))
    else:
        ts_datasets.append(torch.utils.data.TensorDataset(train_X, train_y))

    # Concatenate all datasets into one training set
    training_set = torch.utils.data.ConcatDataset(ts_datasets)
    
    return training_set


class CustomLoss(nn.Module):
    def __init__(self, target_class):
        super(CustomLoss, self).__init__()
        self.target_class = target_class
        self.cross_entropy_loss = nn.CrossEntropyLoss()

    def forward(self, outputs, targets):
        if self.target_class == None:
            filtered_outputs = outputs
            filtered_targets = targets
        else:
            # Filter outputs and targets to keep only the values corresponding to the target_class
            filtered_outputs = torch.zeros_like(outputs)
            filtered_outputs[:, self.target_class] = outputs[:, self.target_class]

            filtered_targets = torch.where(targets == self.target_class, targets, torch.tensor(0, dtype=targets.dtype))

        # Calculate the CrossEntropyLoss using the filtered outputs and targets
        loss = self.cross_entropy_loss(filtered_outputs, filtered_targets)

        return loss


def QuickNN(training_set, n, num_epochs, batch_size, target_class=None,
            l1_lambda = 0.0, stopEarly = 10, visualize=False):
    # Set the device to use (GPU if available, otherwise CPU)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Step 2: Define neural network
    # Instantiate the neural network model
    inputs = training_set[0][0].shape[0]
    model = Net(input_size=inputs, num_classes=n,
                h1_size=n*2, h2_size=n*3, h3_size=n*4, dropout_prob=0
                ).to(device)

    # Define the loss function and optimizer
    criterion = CustomLoss(target_class)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=l1_lambda, amsgrad=True)

    # Define the learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
                                                          factor=0.05, patience=5)
    
    # Step 3: Set up training loop
    train_loader = DataLoader(training_set, batch_size=batch_size, shuffle=True)
    best_loss = float('inf')  # initialize the best validation loss
    early_stop_counter = 0  # initialize the early stopping counter

    if stopEarly > 0:
        print("Early Stopping Initialized")
        # Create the validation set
        val_size = int(len(training_set) * 0.2) # Use 20% of the training set for validation
        val_set, train_set = random_split(training_set, [val_size, len(training_set) - val_size])
        val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True)


    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            if len(inputs) == 0:
                continue
            inputs, targets = inputs.to(device), targets.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            epoch_loss += loss.item()

            # Backward Pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if stopEarly > 0:
            # Define the learning rate scheduler
            lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
                                                                factor=0.05, patience=5)
            # Evaluate the model on the validation set
            with torch.no_grad():
                val_loss = 0.0
                for inputs, targets in val_loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    outputs = model(inputs)
                    val_loss += criterion(outputs, targets).item()
            val_loss /= len(val_loader)

            # Check if the validation loss has improved
            if np.round(val_loss,5) < best_loss:
                best_loss = val_loss
                early_stop_counter = 0
            else:
                early_stop_counter += 1
                if early_stop_counter >= stopEarly:  # if the validation loss hasn't improved for 10 epochs, stop training
                    print(f"Early stopping at: Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Learning rate: {optimizer.param_groups[0]['lr']:.6f}")
                    break
        else:
            # Update the learning rate using the scheduler
            lr_scheduler.step(loss)


        if visualize:
            # Print the training loss and learning rate after every epoch
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Learning rate: {optimizer.param_groups[0]['lr']:.6f}")
        
    return model

def TestModel(test_X, test_y, model, visualize=True):
    # Set the device to use (GPU if available, otherwise CPU)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    test_X = test_X.to(device)
    test_y = test_y.to(device)

    # Evaluate the model on the test set
    with torch.no_grad():
        outputs = model(test_X)
        _, predicted = torch.max(outputs.data, 1)

    results = pd.DataFrame()
    for i in range(min(y), max(y)+1):
        cells = sum(test_y==i).item()
        test_y_i = test_y==i
        y_pred_i = predicted==i
        TP = sum((test_y_i==1) & (y_pred_i==1)).item()
        FP = sum((test_y_i==0) & (y_pred_i==1)).item()
        TN = sum((test_y_i==0) & (y_pred_i==0)).item()
        FN = sum((test_y_i==1) & (y_pred_i==0)).item()
        TPR = TP / np.where(TP+FN == 0, np.nan, TP+FN)
        TNR = TN / np.where(TN+FP == 0, np.nan, TN+FP)
        Prec = TP / np.where(TP+FP == 0, np.nan, TP+FP)
        Accuracy = (TP+TN) / np.where(TP+FP+FN+TN == 0, np.nan, TP+FP+FN+TN)

        res_i = {'Cluster' : le.inverse_transform([i])[0],
            'cells' : cells,
            'TP' : TP,
            'FP' : FP,
            'TN' : TN,
            'FN' : FN,
            'TPR' : TPR,
            'TNR' : TNR,
            'Prec' : Prec,
            'Accuracy' : Accuracy}
        
        res_i = pd.DataFrame([res_i])
        results = pd.concat([results,res_i], ignore_index=True)
        
    if visualize:
        display(results.sort_values(by=['Cluster']))

    return results

def compute_feature_importance(model, input_data, target_category):
    input_data.requires_grad = True # tell PyTorch to compute gradients with respect to the input
    model.zero_grad()
    output = model(input_data)
    # compute the negative log likelihood loss between the output and the target category
    loss = nn.functional.nll_loss(output, target_category) 
    # compute the gradients of the loss with respect to the input.
    loss.backward()
    # feature importance as the mean absolute value of the gradients over the batch dimension (i.e., over all input examples).
    feature_importance = input_data.grad.abs().mean(dim=0)
    return feature_importance.to('cpu')

def unique(gene_list):
    '''This function will remove duplicates in a list while maintaing the order of first appearance'''
    seen = set()
    return [x for x in gene_list if x not in seen and not seen.add(x)]



def gene_ranker(df, all_genes, rank_ordered_genes = [], N=None, show=False):
    '''This function will take a df assuming the column order and row order respectively indicate importance of the gene or cell in question
    the list of all_genes that should ultimately be compared against must also be specified
    rank_ordered_genes is the by default empty list of genes that are being added, however, a list of genes can be provided to fix them at the top
    N is the minimum number of genes that can be returned in the rank orderered gene list
    show will print the final list if desired'''

    # Iterate through columns until rank_ordered_genes contains all_genes
    j = 0
    while set(rank_ordered_genes) != set(all_genes):
        # Get the unique genes in the next column of the input df, that is look at the genes of next importance level
        next_column = unique(list(df.iloc[:, j]))
        # Add the newly found genes to the existing gene list
        rank_ordered_genes.extend(next_column)
        # Remove duplicates while maintaing the order
        rank_ordered_genes = unique(rank_ordered_genes)
        # Iterate to the next column of genes
        j += 1
        # Check if the minimum number of genes desired has been found
        if N != None:
            if len(rank_ordered_genes) >= N:
                break
    if show:
        # Now, rank_ordered_genes contains all the unique genes from reordered_df in the order they appear
        print(rank_ordered_genes)
    return rank_ordered_genes

In [3]:
N = 300 # How many genes to extract
num_epochs = 100 # specify the number of epochs to train for
batch_size = 32 # specify the batch size for training
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


path = '/home/sam/scRNAseq/Xenium/NeurIPS/ML_df.RData'
rdata = pyreadr.read_r(path) # WS
# Load data
df = rdata['combined_df']
del(rdata)
df['Label'] = df['Label'].apply(lambda x: x if len(x.split('_')[0]) == 2 else '0' + x) # Standardize cluster names

# Encode the categoric response 
le = LabelEncoder()
df['Label'] = le.fit_transform(df['Label'])

# Move the response to the end for simply manipulation
cluster_col = df.pop('Label')
df.insert(len(df.columns), 'Cluster', cluster_col)


for seed in range(18,108, 18):
    torch.manual_seed(seed)
    np.random.seed(seed)

    # Shuffle the data
    df_s = df.copy()
    df_s = shuffle(df_s, random_state=seed)

    # Split the data into input features and labels
    X = df_s.iloc[:, :-1].values.astype(np.float32)
    X = np.round(X*100)/100
    y = df_s.iloc[:, -1].values.astype(np.compat.long)
    del(df_s)

    # Convert data to PyTorch tensors
    X = torch.from_numpy(X)
    y = torch.from_numpy(y)

    # Split the data into training and test sets
    train_size = int(0.8 * len(df))
    train_X, test_X = X[:train_size], X[train_size:]
    train_y, test_y = y[:train_size], y[train_size:]

    #############################

    n = len(np.unique(y))

    # Convert train_X and test_X to PyTorch tensors on the GPU
    train_X = train_X.to(device)
    test_X = test_X.to(device)

    training_set = DesiredGeneCurriculum(train_y, train_X, desired=None)
    model = QuickNN(training_set, n, num_epochs, batch_size, stopEarly=0, visualize=False)
    results = TestModel(test_X, test_y, model, visualize=True)

    curriculum_dict = {}
    curriculum_dict[f'Model all final genes'] = {}
    curriculum_dict[f'Model all final genes']['model'] = model

    curriculum_dict[f'Model all final genes']['results'] = results

    # Open a file for writing
    with open(f'/home/sam/scRNAseq/Xenium/NeurIPS/curriculum_models_finalGenes2_{seed}.pkl', 'wb') as f:
        # Use pickle to dump the list to the file
        pickle.dump(curriculum_dict, f)

    results.to_csv(f'/home/sam/scRNAseq/Xenium/NeurIPS/final_gene_list_model_results2_{seed}.csv', index=False)


    # Convert train_X and test_X to PyTorch tensors on the GPU
    train_X = train_X.to(device)
    test_X = test_X.to(device)

    # Convert input data to a PyTorch tensor and move it to the GPU
    input_data = torch.Tensor(test_X).to(device)

    # Compute feature importance for each target category
    mdl = curriculum_dict[f'Model all final genes']['model'].to(device)
    for i in np.unique(y):
        curriculum_dict[f'Model all final genes'][i] = {}
        target_category = torch.full((input_data.shape[0],), i, device=device, dtype=torch.long)
        curriculum_dict[f'Model all final genes'][i]['feature importance'] = compute_feature_importance(mdl, input_data, target_category)
        top_n_values, curriculum_dict[f'Model all final genes'][i][f'top {N} features'] = torch.topk(curriculum_dict[f'Model all final genes'][i]['feature importance'], N, largest=True)
        curriculum_dict[f'Model all final genes'][i][f'top {N} genes'] = list(df.columns[curriculum_dict[f'Model all final genes'][i][f'top {N} features']])


    ordered_gene_list = []
    for i in np.unique(y):
        ordered_gene_list.append([le.inverse_transform([i]), curriculum_dict[f'Model all final genes'][i]['top 300 genes']])

    # Pandas version
    data_dict = {}
    for i in np.unique(y):
        label = le.inverse_transform([i])[0]
        values = curriculum_dict[f'Model all final genes'][i]['top 300 genes']
        data_dict[label] = values

    # Convert the dictionary to a pandas DataFrame
    df_i = pd.DataFrame.from_dict(data_dict, orient='index')
    df_i.index.name = 'Label'

    # Step 1: Create masks for "dd_*" and "AC_*" formats
    dd_mask = df_i.index.str.match(r'^\d{2}_.*')
    ac_mask = df_i.index.str.match(r'^AC_.*')

    # Step 2: Separate rows into three groups
    dd_rows = df_i[dd_mask]
    other_rows = df_i[~(dd_mask | ac_mask)]
    ac_rows = df_i[ac_mask]

    # Step 3: Concatenate the groups to create the reordered DataFrame
    reordered_df = pd.concat([dd_rows, other_rows, ac_rows])

    # Initialize all_genes with the initial list of genes
    all_genes = curriculum_dict[f'Model all final genes'][0]['top 300 genes']

    rank_ordered_genes_byCell = []
    for i in range(1,reordered_df.shape[0]):
        df_subset = reordered_df.iloc[:i,:]
        rank_ordered_genes_byCell = gene_ranker(df_subset, all_genes, rank_ordered_genes_byCell, N=i*3, show=True)
        print("#"*100)
    print(len(rank_ordered_genes_byCell))

    # Open the CSV file in write mode
    csv_file_path = f'/home/sam/scRNAseq/Xenium/NeurIPS/gene_list_order_of_importance_{seed}.csv'
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)

        # Write the list as a single row in the CSV file
        writer.writerow(rank_ordered_genes_byCell)

,Cluster,cells,TP,FP,TN,FN,TPR,TNR,Prec,Accuracy
0,01_W3D1.1,580,579,8,18557,1,0.998276,0.999569,0.986371,0.999530
1,02_W3D1.2,561,555,4,18580,6,0.989305,0.999785,0.992844,0.999478
2,03_FminiON,390,385,1,18754,5,0.987179,0.999947,0.997409,0.999687
3,04_FminiOFF,378,377,6,18761,1,0.997354,0.999680,0.984334,0.999634
4,05_J-RGC,342,338,8,18795,4,0.988304,0.999575,0.976879,0.999373
...,...,...,...,...,...,...,...,...,...,...
122,AC_62,8,2,1,19136,6,0.250000,0.999948,0.666667,0.999634
123,AC_63,5,4,1,19139,1,0.800000,0.999948,0.800000,0.999896
124,AC_7,236,222,24,18885,14,0.940678,0.998731,0.902439,0.998015
125,AC_8,223,214,9,18913,9,0.959641,0.999524,0.959641,0.999060


['Prlr', 'Sptssb', 'Pcdh7']
####################################################################################################
['Prlr', 'Sptssb', 'Pcdh7', 'Ano3', 'Isl2', 'Calb1']
####################################################################################################
['Prlr', 'Sptssb', 'Pcdh7', 'Ano3', 'Isl2', 'Calb1', 'Spp1', 'Tfap2d', 'Tpbg', 'Isl1']
####################################################################################################
['Prlr', 'Sptssb', 'Pcdh7', 'Ano3', 'Isl2', 'Calb1', 'Spp1', 'Tfap2d', 'Tpbg', 'Isl1', 'Zic1', 'Syndig1l']
####################################################################################################
['Prlr', 'Sptssb', 'Pcdh7', 'Ano3', 'Isl2', 'Calb1', 'Spp1', 'Tfap2d', 'Tpbg', 'Isl1', 'Zic1', 'Syndig1l', 'Pou4f3', 'Alcam', 'Kcnh2', 'Necab2']
####################################################################################################
['Prlr', 'Sptssb', 'Pcdh7', 'Ano3', 'Isl2', 'Calb1', 'Spp1', 'Tfap2d', 'Tpb

['Prlr', 'Sptssb', 'Pcdh7', 'Ano3', 'Isl2', 'Calb1', 'Spp1', 'Tfap2d', 'Tpbg', 'Isl1', 'Zic1', 'Syndig1l', 'Pou4f3', 'Alcam', 'Kcnh2', 'Necab2', 'Cck', 'Brinp1', 'Penk', 'Neurod2', 'Tbx20', 'Eomes', 'Chrm2', 'Gal', 'Nmb', 'Syt6', 'Tshz2', 'Etv1', 'Slc17a8', 'Pcdh17', 'Lypd6', 'Nr2f2', 'Igfbp2', 'Kcnip4', 'Neto2', 'Vsx1', 'Pvalb', 'Nr4a2', 'Anxa2', 'Maf', 'Foxp2', 'Pcdh11x', 'Satb1', 'Cartpt', 'Spon1', 'Rprm', 'C1ql3', 'Lypd1', 'Gjd2', 'Vstm2b', 'Ramp3', 'Pcdh9', 'Pcp4', 'Igfbp5', 'Tmeff2', 'Sv2b', 'Meis2', 'Vgf', 'Serpinb1b', 'Fgf11', 'Magi1', 'Lmo4', 'Col12a1', 'Opn4', 'Gsg1', 'Zbtb20', 'Igfbp7', 'Kcnb2', 'Sh3bgr', 'BC048546', 'Ncam2', 'Igf1', 'Gldn', 'Nptx2', 'Pantr1', 'Unc13c', 'Hcn1', 'Ndnf', 'Kcnk2', 'Ppp1r17', 'Cd63', 'Irx3', 'Pou3f1', 'Zfhx3', 'Dkk3', 'Gabrg3', 'Rasgrp1', 'Kcnd2', 'Ebf3', 'Gucy1a3', 'Nrxn3', 'Grin2a', 'Jam2', 'Coch', 'Cpne4', 'Lmo1', 'Irx6', 'Pcdh10', 'Scgn', 'Laptm4b', 'Wscd1', 'Syt2', 'Cd24a', 'Fgf1', 'Scrt1', 'Nefh', 'Cdkn1c', 'C1ql2', 'Slc6a9', 'Ntng1', 'Cpn

['Prlr', 'Sptssb', 'Pcdh7', 'Ano3', 'Isl2', 'Calb1', 'Spp1', 'Tfap2d', 'Tpbg', 'Isl1', 'Zic1', 'Syndig1l', 'Pou4f3', 'Alcam', 'Kcnh2', 'Necab2', 'Cck', 'Brinp1', 'Penk', 'Neurod2', 'Tbx20', 'Eomes', 'Chrm2', 'Gal', 'Nmb', 'Syt6', 'Tshz2', 'Etv1', 'Slc17a8', 'Pcdh17', 'Lypd6', 'Nr2f2', 'Igfbp2', 'Kcnip4', 'Neto2', 'Vsx1', 'Pvalb', 'Nr4a2', 'Anxa2', 'Maf', 'Foxp2', 'Pcdh11x', 'Satb1', 'Cartpt', 'Spon1', 'Rprm', 'C1ql3', 'Lypd1', 'Gjd2', 'Vstm2b', 'Ramp3', 'Pcdh9', 'Pcp4', 'Igfbp5', 'Tmeff2', 'Sv2b', 'Meis2', 'Vgf', 'Serpinb1b', 'Fgf11', 'Magi1', 'Lmo4', 'Col12a1', 'Opn4', 'Gsg1', 'Zbtb20', 'Igfbp7', 'Kcnb2', 'Sh3bgr', 'BC048546', 'Ncam2', 'Igf1', 'Gldn', 'Nptx2', 'Pantr1', 'Unc13c', 'Hcn1', 'Ndnf', 'Kcnk2', 'Ppp1r17', 'Cd63', 'Irx3', 'Pou3f1', 'Zfhx3', 'Dkk3', 'Gabrg3', 'Rasgrp1', 'Kcnd2', 'Ebf3', 'Gucy1a3', 'Nrxn3', 'Grin2a', 'Jam2', 'Coch', 'Cpne4', 'Lmo1', 'Irx6', 'Pcdh10', 'Scgn', 'Laptm4b', 'Wscd1', 'Syt2', 'Cd24a', 'Fgf1', 'Scrt1', 'Nefh', 'Cdkn1c', 'C1ql2', 'Slc6a9', 'Ntng1', 'Cpn

,Cluster,cells,TP,FP,TN,FN,TPR,TNR,Prec,Accuracy
0,01_W3D1.1,606,603,2,18537,3,0.995050,0.999892,0.996694,0.999739
1,02_W3D1.2,589,586,7,18549,3,0.994907,0.999623,0.988196,0.999478
2,03_FminiON,380,376,2,18763,4,0.989474,0.999893,0.994709,0.999687
3,04_FminiOFF,342,337,3,18800,5,0.985380,0.999840,0.991176,0.999582
4,05_J-RGC,324,317,5,18816,7,0.978395,0.999734,0.984472,0.999373
...,...,...,...,...,...,...,...,...,...,...
122,AC_62,6,0,0,19139,6,0.000000,1.000000,NaN,0.999687
123,AC_63,6,5,1,19138,1,0.833333,0.999948,0.833333,0.999896
124,AC_7,276,253,15,18854,23,0.916667,0.999205,0.944030,0.998015
125,AC_8,240,232,6,18899,8,0.966667,0.999683,0.974790,0.999269


...

['Sptssb', 'Prlr', 'Isl2', 'Pvalb', 'Pcdh7', 'Foxp2', 'Isl1', 'Tfap2d', 'Lypd1', 'Gldn', 'Lmo2', 'Nmb', 'Penk', 'Zic1', 'Alcam', 'Ppp1r17', 'Calb1', 'Cd24a', 'Necab2', 'Tbx20', 'Pcdh10', 'Ano3', 'Vstm2b', 'Eomes', 'Pou4f3', 'Ebf3', 'Pou6f2', 'Etv1', 'Pcdh17', 'Maf', 'Slc17a8', 'Cck', 'Kcnip4', 'Gpr88', 'Pcdh9', 'Chrm2', 'Tmeff2', 'Hcn1', 'Lypd6', 'Gal', 'Cdkn1c', 'Spp1', 'Neurod2', 'Brinp1', 'Cpne4', 'Opn4', 'Vsx1', 'Kcnh2', 'Ptgds', 'Gjd2', 'Pcp4', 'Meis2', 'Ncam2', 'B3galt2', 'Serpinb1b', 'Lxn', 'Lmo1', 'Fgf11', 'Syndig1l', 'Spon1', 'Gabrb1', 'Irx3', 'Gsg1', 'Sh3bgr', 'Gucy1a3', 'Gabrg3', 'Coch', 'Cartpt', 'Lmo4', 'Aldoc', 'Kcnk2', 'Grin2a', 'C1ql3', 'Nr4a2', 'Igfbp5', 'Car3', 'Ntng1', 'Serpine2', 'Pcdh11x', 'Syt13', 'Rab3c', 'Rprm', 'Igf1', 'Car2', 'Edil3', 'Scgn', 'Tac1', 'Magi1', 'Igfbp2', 'Ndnf', 'Ramp3', 'Pcp2', 'Car8', 'Snca', 'Nr2f2', 'Nfib', 'Zbtb20', 'Satb1', 'Prkcq', 'Mmp9', 'Tpbg', 'Pantr1', 'Fam19a1', 'Irx6', 'Wscd1', 'Chrnb3', 'Mafb', 'Spc25', 'Ptn', 'Syt6', 'Scg2', 'Cra

,Cluster,cells,TP,FP,TN,FN,TPR,TNR,Prec,Accuracy
0,01_W3D1.1,595,594,4,18546,1,0.998319,0.999784,0.993311,0.999739
1,02_W3D1.2,558,552,8,18579,6,0.989247,0.999570,0.985714,0.999269
2,03_FminiON,404,395,2,18739,9,0.977723,0.999893,0.994962,0.999425
3,04_FminiOFF,376,371,3,18766,5,0.986702,0.999840,0.991979,0.999582
4,05_J-RGC,347,340,3,18795,7,0.979827,0.999840,0.991254,0.999478
...,...,...,...,...,...,...,...,...,...,...
122,AC_62,6,2,4,19135,4,0.333333,0.999791,0.333333,0.999582
123,AC_63,4,4,0,19141,0,1.000000,1.000000,1.000000,1.000000
124,AC_7,227,213,23,18895,14,0.938326,0.998784,0.902542,0.998067
125,AC_8,199,187,7,18939,12,0.939698,0.999631,0.963918,0.999008


['Prlr', 'Kcnip4', 'Sptssb']
####################################################################################################
['Prlr', 'Kcnip4', 'Sptssb', 'Zic1', 'Spp1', 'Isl2', 'Foxp2']
####################################################################################################
['Prlr', 'Kcnip4', 'Sptssb', 'Zic1', 'Spp1', 'Isl2', 'Foxp2', 'Vstm2b', 'Pcdh7', 'Calb1']
####################################################################################################
['Prlr', 'Kcnip4', 'Sptssb', 'Zic1', 'Spp1', 'Isl2', 'Foxp2', 'Vstm2b', 'Pcdh7', 'Calb1', 'Necab2', 'Neurod2']
####################################################################################################
['Prlr', 'Kcnip4', 'Sptssb', 'Zic1', 'Spp1', 'Isl2', 'Foxp2', 'Vstm2b', 'Pcdh7', 'Calb1', 'Necab2', 'Neurod2', 'Tfap2d', 'Alcam', 'Isl1', 'Pou4f3']
####################################################################################################
['Prlr', 'Kcnip4', 'Sptssb', 'Zic1', 'Spp1', 'Isl2', 'F

['Prlr', 'Kcnip4', 'Sptssb', 'Zic1', 'Spp1', 'Isl2', 'Foxp2', 'Vstm2b', 'Pcdh7', 'Calb1', 'Necab2', 'Neurod2', 'Tfap2d', 'Alcam', 'Isl1', 'Pou4f3', 'Serpinb1b', 'Kcnk2', 'Gal', 'Fgf11', 'Gldn', 'Nmb', 'Igfbp5', 'Maf', 'Neto2', 'Opn4', 'Ebf3', 'Tbx20', 'Penk', 'Pcp4', 'Pcdh9', 'Vsx1', 'Etv1', 'Nr2f2', 'Cck', 'Pcdh10', 'Zmat4', 'Gabrg3', 'Rasgrp1', 'Hcn1', 'Pvalb', 'Rprm', 'Lypd6', 'Cd24a', 'Sst', 'Zbtb20', 'Meis2', 'Pcdh17', 'Gsg1', 'Tpbg', 'Irx3', 'Anxa2', 'Scgn', 'Spon1', 'Trp53i11', 'Brinp1', 'Kcnh2', 'Evc2', 'Slc17a8', 'Lmo4', 'Gjd2', 'Cd83', 'Sema6a', 'Lmo1', 'Eomes', 'Coch', 'Mgll', 'Chrm2', 'Tac1', 'Vip', 'Igfbp2', 'Satb1', 'Tshz2', 'Lamp5', 'Sv2b', 'Caln1', 'Pcdh11x', 'Gucy1a3', 'Ncam2', 'Ndnf', 'Gngt1', 'Nr4a2', 'Pou6f2', 'Kcnd2', 'Grin2a', 'Lmo2', 'Ntng1', 'Ano3', 'Syndig1l', 'Pantr1', 'Fgf13', 'Aldoc', 'Qpct', 'Pou3f1', 'Kcnb2', 'Lypd1', 'Npnt', 'Ppp1r17', 'Reln', 'Cabp5', 'Slc1a3', 'Gpr88', 'C1ql3', 'Kcna1', 'Nckap5', 'Thrsp', 'Ptn', 'Cartpt', 'Mgat4c', 'Ptgds', 'Trnp1', 'Cp

['Prlr', 'Kcnip4', 'Sptssb', 'Zic1', 'Spp1', 'Isl2', 'Foxp2', 'Vstm2b', 'Pcdh7', 'Calb1', 'Necab2', 'Neurod2', 'Tfap2d', 'Alcam', 'Isl1', 'Pou4f3', 'Serpinb1b', 'Kcnk2', 'Gal', 'Fgf11', 'Gldn', 'Nmb', 'Igfbp5', 'Maf', 'Neto2', 'Opn4', 'Ebf3', 'Tbx20', 'Penk', 'Pcp4', 'Pcdh9', 'Vsx1', 'Etv1', 'Nr2f2', 'Cck', 'Pcdh10', 'Zmat4', 'Gabrg3', 'Rasgrp1', 'Hcn1', 'Pvalb', 'Rprm', 'Lypd6', 'Cd24a', 'Sst', 'Zbtb20', 'Meis2', 'Pcdh17', 'Gsg1', 'Tpbg', 'Irx3', 'Anxa2', 'Scgn', 'Spon1', 'Trp53i11', 'Brinp1', 'Kcnh2', 'Evc2', 'Slc17a8', 'Lmo4', 'Gjd2', 'Cd83', 'Sema6a', 'Lmo1', 'Eomes', 'Coch', 'Mgll', 'Chrm2', 'Tac1', 'Vip', 'Igfbp2', 'Satb1', 'Tshz2', 'Lamp5', 'Sv2b', 'Caln1', 'Pcdh11x', 'Gucy1a3', 'Ncam2', 'Ndnf', 'Gngt1', 'Nr4a2', 'Pou6f2', 'Kcnd2', 'Grin2a', 'Lmo2', 'Ntng1', 'Ano3', 'Syndig1l', 'Pantr1', 'Fgf13', 'Aldoc', 'Qpct', 'Pou3f1', 'Kcnb2', 'Lypd1', 'Npnt', 'Ppp1r17', 'Reln', 'Cabp5', 'Slc1a3', 'Gpr88', 'C1ql3', 'Kcna1', 'Nckap5', 'Thrsp', 'Ptn', 'Cartpt', 'Mgat4c', 'Ptgds', 'Trnp1', 'Cp

In [20]:
seed_dict = {}
for seed in range(18,108, 18):
    torch.manual_seed(seed)
    np.random.seed(seed)

    # Shuffle the data
    csv_file_path = f'/home/sam/scRNAseq/Xenium/NeurIPS/gene_list_order_of_importance_{seed}.csv'
    columns_df = pd.read_csv(csv_file_path)
    column_names = list(columns_df.columns)
    column_names = column_names[0:300]
    column_names.append('Cluster')
    df_s = df.copy()[column_names]
    df_s = shuffle(df_s, random_state=seed)

    # Split the data into input features and labels
    X = df_s.iloc[:, :-1].values.astype(np.float32)
    X = np.round(X*100)/100
    y = df_s.iloc[:, -1].values.astype(np.compat.long)
    del(df_s)

    # Convert data to PyTorch tensors
    X = torch.from_numpy(X)
    y = torch.from_numpy(y)

    # Split the data into training and test sets
    train_size = int(0.8 * len(df))
    train_X, test_X = X[:train_size], X[train_size:]
    train_y, test_y = y[:train_size], y[train_size:]
    
    n = len(np.unique(y))

    # Convert train_X and test_X to PyTorch tensors on the GPU
    train_X = train_X.to(device)
    test_X = test_X.to(device)

    training_set = DesiredGeneCurriculum(train_y, train_X, desired=None)
    model = QuickNN(training_set, n, num_epochs, batch_size, stopEarly=0, visualize=False)
    results = TestModel(test_X, test_y, model, visualize=True)


    seed_dict[seed] = {}
    seed_dict[seed]['model'] = model
    seed_dict[seed]['results'] = results
    seed_dict[seed]['genes'] = column_names[0:300]

    results.to_csv(f'/home/sam/scRNAseq/Xenium/NeurIPS/GradientAlgo_results_{seed}.csv', index=False)
# Open a file for writing
with open(f'/home/sam/scRNAseq/Xenium/NeurIPS/All_Variants_GradientAlgo.pkl', 'wb') as f:
    # Use pickle to dump the list to the file
    pickle.dump(seed_dict, f)

,Cluster,cells,TP,FP,TN,FN,TPR,TNR,Prec,Accuracy
0,01_W3D1.1,599,595,2,18544,4,0.993322,0.999892,0.996650,0.999687
1,02_W3D1.2,570,558,7,18568,12,0.978947,0.999623,0.987611,0.999008
2,03_FminiON,412,409,8,18725,3,0.992718,0.999573,0.980815,0.999425
3,04_FminiOFF,389,385,4,18752,4,0.989717,0.999787,0.989717,0.999582
4,05_J-RGC,340,332,1,18804,8,0.976471,0.999947,0.996997,0.999530
...,...,...,...,...,...,...,...,...,...,...
122,AC_62,4,1,1,19140,3,0.250000,0.999948,0.500000,0.999791
123,AC_63,5,3,1,19139,2,0.600000,0.999948,0.750000,0.999843
124,AC_7,264,244,21,18860,20,0.924242,0.998888,0.920755,0.997858
125,AC_8,214,206,13,18918,8,0.962617,0.999313,0.940639,0.998903


,Cluster,cells,TP,FP,TN,FN,TPR,TNR,Prec,Accuracy
0,01_W3D1.1,580,579,5,18560,1,0.998276,0.999731,0.991438,0.999687
1,02_W3D1.2,561,554,4,18580,7,0.987522,0.999785,0.992832,0.999425
2,03_FminiON,390,384,4,18751,6,0.984615,0.999787,0.989691,0.999478
3,04_FminiOFF,378,377,7,18760,1,0.997354,0.999627,0.981771,0.999582
4,05_J-RGC,342,337,12,18791,5,0.985380,0.999362,0.965616,0.999112
...,...,...,...,...,...,...,...,...,...,...
122,AC_62,8,2,4,19133,6,0.250000,0.999791,0.333333,0.999478
123,AC_63,5,5,1,19139,0,1.000000,0.999948,0.833333,0.999948
124,AC_7,236,222,17,18892,14,0.940678,0.999101,0.928870,0.998381
125,AC_8,223,209,10,18912,14,0.937220,0.999472,0.954338,0.998746


,Cluster,cells,TP,FP,TN,FN,TPR,TNR,Prec,Accuracy
0,01_W3D1.1,606,600,4,18535,6,0.990099,0.999784,0.993377,0.999478
1,02_W3D1.2,589,580,8,18548,9,0.984720,0.999569,0.986395,0.999112
2,03_FminiON,380,376,4,18761,4,0.989474,0.999787,0.989474,0.999582
3,04_FminiOFF,342,339,7,18796,3,0.991228,0.999628,0.979769,0.999478
4,05_J-RGC,324,313,10,18811,11,0.966049,0.999469,0.969040,0.998903
...,...,...,...,...,...,...,...,...,...,...
122,AC_62,6,0,0,19139,6,0.000000,1.000000,NaN,0.999687
123,AC_63,6,5,1,19138,1,0.833333,0.999948,0.833333,0.999896
124,AC_7,276,256,20,18849,20,0.927536,0.998940,0.927536,0.997911
125,AC_8,240,233,12,18893,7,0.970833,0.999365,0.951020,0.999008


,Cluster,cells,TP,FP,TN,FN,TPR,TNR,Prec,Accuracy
0,01_W3D1.1,603,599,6,18536,4,0.993367,0.999676,0.990083,0.999478
1,02_W3D1.2,527,520,9,18609,7,0.986717,0.999517,0.982987,0.999164
2,03_FminiON,390,382,7,18748,8,0.979487,0.999627,0.982005,0.999217
3,04_FminiOFF,364,359,4,18777,5,0.986264,0.999787,0.988981,0.999530
4,05_J-RGC,338,334,6,18801,4,0.988166,0.999681,0.982353,0.999478
...,...,...,...,...,...,...,...,...,...,...
122,AC_62,8,2,2,19135,6,0.250000,0.999895,0.500000,0.999582
123,AC_63,6,6,0,19139,0,1.000000,1.000000,1.000000,1.000000
124,AC_7,231,210,20,18894,21,0.909091,0.998943,0.913043,0.997858
125,AC_8,233,224,7,18905,9,0.961373,0.999630,0.969697,0.999164


,Cluster,cells,TP,FP,TN,FN,TPR,TNR,Prec,Accuracy
0,01_W3D1.1,595,593,5,18545,2,0.996639,0.999730,0.991639,0.999634
1,02_W3D1.2,558,550,8,18579,8,0.985663,0.999570,0.985663,0.999164
2,03_FminiON,404,394,6,18735,10,0.975248,0.999680,0.985000,0.999164
3,04_FminiOFF,376,371,5,18764,5,0.986702,0.999734,0.986702,0.999478
4,05_J-RGC,347,340,3,18795,7,0.979827,0.999840,0.991254,0.999478
...,...,...,...,...,...,...,...,...,...,...
122,AC_62,6,2,7,19132,4,0.333333,0.999634,0.222222,0.999425
123,AC_63,4,4,0,19141,0,1.000000,1.000000,1.000000,1.000000
124,AC_7,227,209,23,18895,18,0.920705,0.998784,0.900862,0.997858
125,AC_8,199,186,8,18938,13,0.934673,0.999578,0.958763,0.998903


In [23]:
thresh = 0.9
F1s = []
for seed in range(18,108, 18):
    results = seed_dict[seed]['results']
    # Create a new column that assigns 1 if both TPR and Prec are above the threshold, otherwise 0
    results['Above_Threshold'] = ((results['TPR'] >= thresh) & (results['Prec'] >= thresh)).astype(int)
    
    # Calculate the sum of the 'Above_Threshold' column
    count_above_thresh = results['Above_Threshold'].sum()
    
    # Append the count to the list
    F1s.append(count_above_thresh)

print("Counts of rows with TPR and Prec above 0.9:", F1s)
    
# Convert the list to a DataFrame
df_f1s = pd.DataFrame({'Counts': F1s})

# Save the DataFrame to a CSV file
df_f1s.to_csv('/home/sam/scRNAseq/Xenium/NeurIPS/All_Variants_GradientAlgo_thresh_summary.csv', index=False)

Counts of rows with TPR and Prec above 0.9: [92, 91, 92, 93, 95]


In [31]:
# Initialize an empty list to hold dictionaries
results_list = []
seed_dict={}

for j in range(50, 325, 25):
    seed_dict[j] = {}
    for seed in range(18, 108, 18):
        torch.manual_seed(seed)
        np.random.seed(seed)

        # Shuffle and process the data
        csv_file_path = f'/home/sam/scRNAseq/Xenium/NeurIPS/gene_list_order_of_importance_{seed}.csv'
        columns_df = pd.read_csv(csv_file_path)
        column_names = list(columns_df.columns)[:j] + ['Cluster']
        df_s = df.copy()[column_names]
        df_s = shuffle(df_s, random_state=seed)

        X = df_s.iloc[:, :-1].values.astype(np.float32)
        X = np.round(X*100)/100
        y = df_s.iloc[:, -1].values.astype(np.compat.long)
        del df_s

        X = torch.from_numpy(X)
        y = torch.from_numpy(y)

        train_size = int(0.8 * len(df))
        train_X, test_X = X[:train_size], X[train_size:]
        train_y, test_y = y[:train_size], y[train_size:]

        train_X = train_X.to(device)
        test_X = test_X.to(device)

        training_set = DesiredGeneCurriculum(train_y, train_X, desired=None)
        model = QuickNN(training_set, len(np.unique(y)), num_epochs, batch_size, stopEarly=0, visualize=False)
        results = TestModel(test_X, test_y, model, visualize=True)

        seed_dict[j][seed] = {'model': model, 'results': results, 'genes': column_names[:-1]}
        results.to_csv(f'/home/sam/scRNAseq/Xenium/NeurIPS/GradientAlgo_results_{seed}.csv', index=False)

    
    F1s = []
    for seed in range(18, 108, 18):
        results = seed_dict[j][seed]['results']
        for thresh in [0.75, 0.8, 0.85, 0.9, 0.95]:
            results['Above_Threshold'] = ((results['TPR'] >= thresh) & (results['Prec'] >= thresh)).astype(int)
            count_above_thresh = results['Above_Threshold'].sum()

            # Append the count along with 'seed' and 'j' to the list
            results_list.append({'Seed': seed, 'N_genes': j, "Thresh" : thresh, 'Counts': count_above_thresh})

# Convert the list of dictionaries to a DataFrame
df_f1s = pd.DataFrame(results_list)

# Save the DataFrame to a CSV file
df_f1s.to_csv('/home/sam/scRNAseq/Xenium/NeurIPS/All_Variants_GradientAlgo_thresh_summary.csv', index=False)


# Open a file for writing
with open(f'/home/sam/scRNAseq/Xenium/NeurIPS/All_Variants_GradientAlgo.pkl', 'wb') as f:
    # Use pickle to dump the list to the file
    pickle.dump(seed_dict, f)

,Cluster,cells,TP,FP,TN,FN,TPR,TNR,Prec,Accuracy
0,01_W3D1.1,599,588,15,18531,11,0.981636,0.999191,0.975124,0.998642
1,02_W3D1.2,570,552,8,18567,18,0.968421,0.999569,0.985714,0.998642
2,03_FminiON,412,404,22,18711,8,0.980583,0.998826,0.948357,0.998433
3,04_FminiOFF,389,374,14,18742,15,0.961440,0.999254,0.963918,0.998485
4,05_J-RGC,340,308,31,18774,32,0.905882,0.998352,0.908555,0.996709
...,...,...,...,...,...,...,...,...,...,...
122,AC_62,4,0,3,19138,4,0.000000,0.999843,0.000000,0.999634
123,AC_63,5,2,1,19139,3,0.400000,0.999948,0.666667,0.999791
124,AC_7,264,141,114,18767,123,0.534091,0.993962,0.552941,0.987621
125,AC_8,214,134,68,18863,80,0.626168,0.996408,0.663366,0.992270


,Cluster,cells,TP,FP,TN,FN,TPR,TNR,Prec,Accuracy
0,01_W3D1.1,580,574,20,18545,6,0.989655,0.998923,0.966330,0.998642
1,02_W3D1.2,561,544,15,18569,17,0.969697,0.999193,0.973166,0.998329
2,03_FminiON,390,377,6,18749,13,0.966667,0.999680,0.984334,0.999008
3,04_FminiOFF,378,368,11,18756,10,0.973545,0.999414,0.970976,0.998903
4,05_J-RGC,342,315,38,18765,27,0.921053,0.997979,0.892351,0.996605
...,...,...,...,...,...,...,...,...,...,...
122,AC_62,8,0,0,19137,8,0.000000,1.000000,NaN,0.999582
123,AC_63,5,3,2,19138,2,0.600000,0.999896,0.600000,0.999791
124,AC_7,236,125,79,18830,111,0.529661,0.995822,0.612745,0.990076
125,AC_8,223,149,59,18863,74,0.668161,0.996882,0.716346,0.993053


,Cluster,cells,TP,FP,TN,FN,TPR,TNR,Prec,Accuracy
0,01_W3D1.1,606,593,11,18528,13,0.978548,0.999407,0.981788,0.998746
1,02_W3D1.2,589,576,17,18539,13,0.977929,0.999084,0.971332,0.998433
2,03_FminiON,380,367,16,18749,13,0.965789,0.999147,0.958225,0.998485
3,04_FminiOFF,342,323,17,18786,19,0.944444,0.999096,0.950000,0.998120
4,05_J-RGC,324,294,29,18792,30,0.907407,0.998459,0.910217,0.996918
...,...,...,...,...,...,...,...,...,...,...
122,AC_62,6,0,0,19139,6,0.000000,1.000000,NaN,0.999687
123,AC_63,6,2,3,19136,4,0.333333,0.999843,0.400000,0.999634
124,AC_7,276,168,95,18774,108,0.608696,0.994965,0.638783,0.989397
125,AC_8,240,141,77,18828,99,0.587500,0.995927,0.646789,0.990807


,Cluster,cells,TP,FP,TN,FN,TPR,TNR,Prec,Accuracy
0,01_W3D1.1,603,592,8,18534,11,0.981758,0.999569,0.986667,0.999008
1,02_W3D1.2,527,514,16,18602,13,0.975332,0.999141,0.969811,0.998485
2,03_FminiON,390,378,17,18738,12,0.969231,0.999094,0.956962,0.998485
3,04_FminiOFF,364,348,25,18756,16,0.956044,0.998669,0.932976,0.997858
4,05_J-RGC,338,299,42,18765,39,0.884615,0.997767,0.876833,0.995769
...,...,...,...,...,...,...,...,...,...,...
122,AC_62,8,0,0,19137,8,0.000000,1.000000,NaN,0.999582
123,AC_63,6,3,2,19137,3,0.500000,0.999896,0.600000,0.999739
124,AC_7,231,142,121,18793,89,0.614719,0.993603,0.539924,0.989031
125,AC_8,233,160,35,18877,73,0.686695,0.998149,0.820513,0.994359


,Cluster,cells,TP,FP,TN,FN,TPR,TNR,Prec,Accuracy
0,01_W3D1.1,595,583,17,18533,12,0.979832,0.999084,0.971667,0.998485
1,02_W3D1.2,558,530,30,18557,28,0.949821,0.998386,0.946429,0.996970
2,03_FminiON,404,385,16,18725,19,0.952970,0.999146,0.960100,0.998172
3,04_FminiOFF,376,368,14,18755,8,0.978723,0.999254,0.963351,0.998851
4,05_J-RGC,347,320,19,18779,27,0.922190,0.998989,0.943953,0.997597
...,...,...,...,...,...,...,...,...,...,...
122,AC_62,6,0,3,19136,6,0.000000,0.999843,0.000000,0.999530
123,AC_63,4,4,0,19141,0,1.000000,1.000000,1.000000,1.000000
124,AC_7,227,149,76,18842,78,0.656388,0.995983,0.662222,0.991956
125,AC_8,199,126,83,18863,73,0.633166,0.995619,0.602871,0.991852


...

,Cluster,cells,TP,FP,TN,FN,TPR,TNR,Prec,Accuracy
0,01_W3D1.1,599,595,2,18544,4,0.993322,0.999892,0.996650,0.999687
1,02_W3D1.2,570,558,7,18568,12,0.978947,0.999623,0.987611,0.999008
2,03_FminiON,412,409,8,18725,3,0.992718,0.999573,0.980815,0.999425
3,04_FminiOFF,389,385,4,18752,4,0.989717,0.999787,0.989717,0.999582
4,05_J-RGC,340,332,1,18804,8,0.976471,0.999947,0.996997,0.999530
...,...,...,...,...,...,...,...,...,...,...
122,AC_62,4,1,1,19140,3,0.250000,0.999948,0.500000,0.999791
123,AC_63,5,3,1,19139,2,0.600000,0.999948,0.750000,0.999843
124,AC_7,264,244,21,18860,20,0.924242,0.998888,0.920755,0.997858
125,AC_8,214,206,13,18918,8,0.962617,0.999313,0.940639,0.998903


,Cluster,cells,TP,FP,TN,FN,TPR,TNR,Prec,Accuracy
0,01_W3D1.1,580,579,5,18560,1,0.998276,0.999731,0.991438,0.999687
1,02_W3D1.2,561,554,4,18580,7,0.987522,0.999785,0.992832,0.999425
2,03_FminiON,390,384,4,18751,6,0.984615,0.999787,0.989691,0.999478
3,04_FminiOFF,378,377,7,18760,1,0.997354,0.999627,0.981771,0.999582
4,05_J-RGC,342,337,12,18791,5,0.985380,0.999362,0.965616,0.999112
...,...,...,...,...,...,...,...,...,...,...
122,AC_62,8,2,4,19133,6,0.250000,0.999791,0.333333,0.999478
123,AC_63,5,5,1,19139,0,1.000000,0.999948,0.833333,0.999948
124,AC_7,236,222,17,18892,14,0.940678,0.999101,0.928870,0.998381
125,AC_8,223,209,10,18912,14,0.937220,0.999472,0.954338,0.998746


,Cluster,cells,TP,FP,TN,FN,TPR,TNR,Prec,Accuracy
0,01_W3D1.1,606,600,4,18535,6,0.990099,0.999784,0.993377,0.999478
1,02_W3D1.2,589,580,8,18548,9,0.984720,0.999569,0.986395,0.999112
2,03_FminiON,380,376,4,18761,4,0.989474,0.999787,0.989474,0.999582
3,04_FminiOFF,342,339,7,18796,3,0.991228,0.999628,0.979769,0.999478
4,05_J-RGC,324,313,10,18811,11,0.966049,0.999469,0.969040,0.998903
...,...,...,...,...,...,...,...,...,...,...
122,AC_62,6,0,0,19139,6,0.000000,1.000000,NaN,0.999687
123,AC_63,6,5,1,19138,1,0.833333,0.999948,0.833333,0.999896
124,AC_7,276,256,20,18849,20,0.927536,0.998940,0.927536,0.997911
125,AC_8,240,233,12,18893,7,0.970833,0.999365,0.951020,0.999008


,Cluster,cells,TP,FP,TN,FN,TPR,TNR,Prec,Accuracy
0,01_W3D1.1,603,599,6,18536,4,0.993367,0.999676,0.990083,0.999478
1,02_W3D1.2,527,520,9,18609,7,0.986717,0.999517,0.982987,0.999164
2,03_FminiON,390,382,7,18748,8,0.979487,0.999627,0.982005,0.999217
3,04_FminiOFF,364,359,4,18777,5,0.986264,0.999787,0.988981,0.999530
4,05_J-RGC,338,334,6,18801,4,0.988166,0.999681,0.982353,0.999478
...,...,...,...,...,...,...,...,...,...,...
122,AC_62,8,2,2,19135,6,0.250000,0.999895,0.500000,0.999582
123,AC_63,6,6,0,19139,0,1.000000,1.000000,1.000000,1.000000
124,AC_7,231,210,20,18894,21,0.909091,0.998943,0.913043,0.997858
125,AC_8,233,224,7,18905,9,0.961373,0.999630,0.969697,0.999164


,Cluster,cells,TP,FP,TN,FN,TPR,TNR,Prec,Accuracy
0,01_W3D1.1,595,593,5,18545,2,0.996639,0.999730,0.991639,0.999634
1,02_W3D1.2,558,550,8,18579,8,0.985663,0.999570,0.985663,0.999164
2,03_FminiON,404,394,6,18735,10,0.975248,0.999680,0.985000,0.999164
3,04_FminiOFF,376,371,5,18764,5,0.986702,0.999734,0.986702,0.999478
4,05_J-RGC,347,340,3,18795,7,0.979827,0.999840,0.991254,0.999478
...,...,...,...,...,...,...,...,...,...,...
122,AC_62,6,2,7,19132,4,0.333333,0.999634,0.222222,0.999425
123,AC_63,4,4,0,19141,0,1.000000,1.000000,1.000000,1.000000
124,AC_7,227,209,23,18895,18,0.920705,0.998784,0.900862,0.997858
125,AC_8,199,186,8,18938,13,0.934673,0.999578,0.958763,0.998903


In [32]:
df_f1s

,Seed,N_genes,Thresh,Counts
0,18,50,0.75,57
1,18,50,0.80,51
2,18,50,0.85,44
3,18,50,0.90,30
4,18,50,0.95,8
...,...,...,...,...
270,90,300,0.75,116
271,90,300,0.80,114
272,90,300,0.85,107
273,90,300,0.90,95
